In [9]:
from scripts.game_components import  ConnectFourBoard, ConnectFourGame, ComputerPlayer

import pandas as pd 
import numpy as np
import tensorflow as tf
import random

In [37]:
%%time
n_games = 10_000
game_history_df_list = []

board_history_list = []
for i in range(n_games):
    board = ConnectFourBoard()
    player1 = ComputerPlayer(board)
    player2 = ComputerPlayer(board)
    game = ConnectFourGame(board, player1, player2)
    game()
    game.history.history['winner'] = game.board.winner
    game.history.history['game_number'] = i
    game_history_df_list.append(game.history.history)

game_history_df = pd.concat(game_history_df_list)


CPU times: total: 12 s
Wall time: 12 s


In [38]:
def reconstruct_board(game_number):
    move_data = (
        game_history_df
        [game_history_df['game_number'] == game_number]
        [['player', 'row', 'col']]
    )
    winner = game_history_df[game_history_df['game_number'] == game_number]['winner'].iloc[0]
    player1_wins = winner == 1
    board_list = []
    board = ConnectFourBoard()
    for _, move in move_data.iterrows():
        board.drop_piece(move['col'], move['player'])
        board_list.append(board.board.copy())
    return board_list, player1_wins

In [39]:
full_board_list = []
player1_wins_list = []
for i in range(n_games):
    board_list, player1_wins = reconstruct_board(i)
    full_board_list.extend(board_list)
    player1_wins_list.extend([player1_wins] * len(board_list))

In [40]:
full_board_array = np.stack(full_board_list, axis=0)
player1_wins_array = np.array(player1_wins_list)
print(full_board_array.shape, player1_wins_array.shape)

(212984, 6, 7) (212984,)


In [41]:
num_actions = 7
num_states = [6, 7]


input = tf.keras.layers.Input(shape = (6, 7))
flatten = tf.keras.layers.Flatten()(input)
hidden_1 = tf.keras.layers.Dense(32, activation = 'relu')(flatten)
hidden_2 = tf.keras.layers.Dense(16, activation = 'relu')(hidden_1)
# output = tf.keras.layers.Dense(num_actions, activation = "linear")(hidden_2) 
output = tf.keras.layers.Dense(1, activation = "sigmoid")(hidden_2)
c4_model = tf.keras.models.Model(inputs = [input], outputs = [output])

In [42]:
c4_model.compile(optimizer='adam',
                 loss='binary_crossentropy',  # or another appropriate loss function depending on your problem
                 metrics=['accuracy'])       # you can change this metric based on your problem


In [43]:
history = c4_model.fit(full_board_array, player1_wins_array, epochs=10, batch_size=32, validation_split=0.2)


Epoch 1/10
5325/5325 [==============================] - 6s 1ms/step - loss: 0.6785 - accuracy: 0.5724 - val_loss: 0.6877 - val_accuracy: 0.5557
Epoch 2/10
5325/5325 [==============================] - 6s 1ms/step - loss: 0.6674 - accuracy: 0.5926 - val_loss: 0.6946 - val_accuracy: 0.5475
Epoch 3/10
5325/5325 [==============================] - 6s 1ms/step - loss: 0.6613 - accuracy: 0.6023 - val_loss: 0.6907 - val_accuracy: 0.5564
Epoch 4/10
5325/5325 [==============================] - 6s 1ms/step - loss: 0.6568 - accuracy: 0.6076 - val_loss: 0.6938 - val_accuracy: 0.5600
Epoch 5/10
5325/5325 [==============================] - 6s 1ms/step - loss: 0.6531 - accuracy: 0.6126 - val_loss: 0.7007 - val_accuracy: 0.5609
Epoch 6/10
5325/5325 [==============================] - 6s 1ms/step - loss: 0.6503 - accuracy: 0.6149 - val_loss: 0.7012 - val_accuracy: 0.5579
Epoch 7/10
5325/5325 [==============================] - 6s 1ms/step - loss: 0.6476 - accuracy: 0.6182 - val_loss: 0.7022 - val_accuracy:

In [6]:
winner_counts = (
    game_history_df
    [['winner', 'game_number']]
    .drop_duplicates()
    .value_counts('winner')
    .reset_index()
)
winner_counts

,winner,count
0,1,561
1,2,435


In [11]:
#Define epsilon decision function
def epsilonDecision(epsilon):
  action_decision = random.choices(['model','random'], weights = [1 - epsilon, epsilon])[0]
  return action_decision
epsilonDecision(epsilon = 0.10) # would always give 'mode

'model'

In [19]:
num_actions = 7
num_states = [6, 7]


input = tf.keras.layers.Input(shape = (6, 7))
flatten = tf.keras.layers.Flatten()(input)
hidden_1 = tf.keras.layers.Dense(50, activation = 'relu')(flatten)
hidden_2 = tf.keras.layers.Dense(50, activation = 'relu')(hidden_1)
hidden_3 = tf.keras.layers.Dense(50, activation = 'relu')(hidden_2)
hidden_4 = tf.keras.layers.Dense(50, activation = 'relu')(hidden_3)
output = tf.keras.layers.Dense(num_actions, activation = "linear")(hidden_4) 
c4_model = tf.keras.models.Model(inputs = [input], outputs = [output])

In [24]:
len(full_board_list)

22005